# **See the speaker folder**

In [ ]:
import os

base_path = '/content/drive/MyDrive/ears data'
print("Contents of ears data folder:")
print(os.listdir(base_path))

# **Resampled to 16KHz and fixed length to 15 second**

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np

# --- Parameters ---
root_dir = "/content/drive/MyDrive/ears data"  # Original dataset folder containing p001, p002, ...
output_dir = "/content/drive/MyDrive/ears_data_15s"  # Folder to save processed audio
target_sr = 16000
target_duration = 15  # seconds
target_length = target_sr * target_duration  # 15*16000 = 240000 samples

# Create output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# --- Process each speaker folder ---
for speaker in os.listdir(root_dir):
    speaker_path = os.path.join(root_dir, speaker)
    if not os.path.isdir(speaker_path):
        continue

    # Create corresponding output folder for speaker
    out_speaker_path = os.path.join(output_dir, speaker)
    os.makedirs(out_speaker_path, exist_ok=True)

    # Process each audio file
    for file_name in os.listdir(speaker_path):
        if not file_name.endswith('.wav'):
            continue

        file_path = os.path.join(speaker_path, file_name)

        # Load audio and resample to 16kHz
        audio, sr = librosa.load(file_path, sr=target_sr)

        # Pad or truncate to 15 seconds
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
        else:
            audio = audio[:target_length]

        # Save the processed audio
        out_file_path = os.path.join(out_speaker_path, file_name)
        sf.write(out_file_path, audio, target_sr)

    print(f"Processed all files for speaker {speaker}")



# **MFCC extraction(.npy file)**

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

# --- Paths ---
audio_dir = "/content/drive/MyDrive/ears_data_15s"
output_csv = "/content/drive/MyDrive/ears_data/features_mfcc_segments.csv"
output_npy_dir = "/content/drive/MyDrive/ears_data/mfcc_segments_npy"

# --- Parameters ---
SAMPLE_RATE = 16000
PRE_EMPHASIS = 0.97
N_MFCC = 13
FRAME_SIZE = 469     # frames per segment
FRAME_OVERLAP = 128  # overlapping frames

# Create output directory
os.makedirs(output_npy_dir, exist_ok=True)

# --- Helper Functions ---
def pre_emphasis(signal, alpha=0.97):
    """Apply pre-emphasis to boost high frequencies."""
    return np.append(signal[0], signal[1:] - alpha * signal[:-1])

def segment_mfcc(mfcc, frame_size, overlap):
    """Segment MFCC matrix into overlapping windows."""
    step = frame_size - overlap
    segments = []
    for start in range(0, mfcc.shape[1] - frame_size + 1, step):
        seg = mfcc[:, start:start + frame_size]
        segments.append(seg)
    return np.array(segments)

# --- Extraction ---
metadata = []

for speaker in sorted(os.listdir(audio_dir)):
    speaker_path = os.path.join(audio_dir, speaker)
    if not os.path.isdir(speaker_path):
        continue

    print(f"\n🔹 Extracting MFCC segments for speaker: {speaker}")
    for file_name in tqdm(os.listdir(speaker_path)):
        if not file_name.endswith(".wav"):
            continue

        file_path = os.path.join(speaker_path, file_name)
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        y = pre_emphasis(y, alpha=PRE_EMPHASIS)

        # Compute MFCCs (13 coefficients)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)

        # Segment into overlapping windows
        segments = segment_mfcc(mfcc, FRAME_SIZE, FRAME_OVERLAP)

        # Save each segment as a .npy file
        for seg_idx, seg in enumerate(segments):
            seg_filename = f"{speaker}_{os.path.splitext(file_name)[0]}_seg{seg_idx}.npy"
            seg_path = os.path.join(output_npy_dir, seg_filename)
            np.save(seg_path, seg)

            metadata.append({
                "speaker": speaker,
                "file": file_name,
                "segment_index": seg_idx,
                "npy_path": seg_path
            })

# --- Save metadata ---
df = pd.DataFrame(metadata)
df.to_csv(output_csv, index=False)
print(f"\n✅ MFCC segmentation complete!")
print(f"Saved segment metadata to: {output_csv}")
print(f"Saved .npy MFCC segments to: {output_npy_dir}")


# **Mel Spectogram & Chroma features extraction**

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
from tqdm import tqdm

# === Set the speaker folder you want to process ===
SPEAKER_ID = "p098"  # 🔹 change this for each run (e.g., p002, p003, ...)

# === Paths ===
AUDIO_DIR = f"/content/drive/MyDrive/ears_data_15s/{SPEAKER_ID}"
OUTPUT_MEL_DIR = f"/content/drive/MyDrive/ears_data/features_mel_spectrograms/{SPEAKER_ID}"
OUTPUT_CHROMA_DIR = f"/content/drive/MyDrive/ears_data/features_chroma/{SPEAKER_ID}"
OUTPUT_MEL_IMG_DIR = f"/content/drive/MyDrive/ears_data/features_mel_images/{SPEAKER_ID}"

# === Parameters ===
SAMPLE_RATE = 16000
N_MELS = 128
N_CHROMA = 12
N_FFT = 1024
HOP_LENGTH = 512

# === Create output folders ===
os.makedirs(OUTPUT_MEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_CHROMA_DIR, exist_ok=True)
os.makedirs(OUTPUT_MEL_IMG_DIR, exist_ok=True)

print(f"🎧 Extracting features for: {SPEAKER_ID}")

# === Feature extraction loop ===
for file_name in tqdm(os.listdir(AUDIO_DIR)):
    if not file_name.endswith(".wav"):
        continue

    file_path = os.path.join(AUDIO_DIR, file_name)

    # === Load audio ===
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE)

    # === 1. Mel-Spectrogram ===
    mel_spec = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=2.0
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # === Save as .npy ===
    mel_save_path = os.path.join(OUTPUT_MEL_DIR, file_name.replace(".wav", "_mel.npy"))
    np.save(mel_save_path, mel_spec_db)

    # === Save as PNG Image ===
    mel_img_path = os.path.join(OUTPUT_MEL_IMG_DIR, file_name.replace(".wav", "_mel.png"))
    plt.figure(figsize=(4, 4))
    librosa.display.specshow(
        mel_spec_db,
        sr=sr,
        hop_length=HOP_LENGTH,
        x_axis='time',
        y_axis='mel',
        cmap='magma'
    )
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(mel_img_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    # === 2. Chroma features ===
    chroma = librosa.feature.chroma_stft(
        y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_chroma=N_CHROMA
    )
    chroma_save_path = os.path.join(OUTPUT_CHROMA_DIR, file_name.replace(".wav", "_chroma.npy"))
    np.save(chroma_save_path, chroma)

print(f"✅ Completed feature extraction for {SPEAKER_ID}")
print(f"📁 Mel images saved to: {OUTPUT_MEL_IMG_DIR}")


# **Proposed Model( 5 emotions)**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, MaxPooling2D,
                                     GlobalAveragePooling2D, Dense, Dropout, Flatten,Concatenate)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder, StandardScaler


import random

def time_mask(spec, max_mask_size=20):
    spec = spec.copy()
    t = spec.shape[0]
    mask = random.randint(0, max_mask_size)
    t0 = random.randint(0, max(1, t - mask))
    spec[t0:t0 + mask, :] = 0
    return spec

def freq_mask(spec, max_mask_size=10):
    spec = spec.copy()
    f = spec.shape[1]
    mask = random.randint(0, max_mask_size)
    f0 = random.randint(0, max(1, f - mask))
    spec[:, f0:f0 + mask] = 0
    return spec

def gaussian_noise(spec, noise_std=0.01):
    return spec + np.random.normal(0, noise_std, spec.shape)

def random_augment(spec):
    """Apply 1–3 random augmentations."""
    aug = spec.copy()
    ops = []

    # randomly choose augmentation operations
    if random.random() < 0.7:
        ops.append(time_mask)
    if random.random() < 0.7:
        ops.append(freq_mask)
    if random.random() < 0.5:
        ops.append(gaussian_noise)

    # apply operations randomly
    random.shuffle(ops)
    for op in ops:
        aug = op(aug)
    return aug

# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
DATA_ROOT = '../input/spect-image/features_mel_image23' #spectogram
npy_dir = '../input/ears-ser/mfcc_segments_23emotions' #MFCC
selected_emotions = ["adoration", "anger", "fear", "neutral", "sadness"]

IMG_SIZE = (128, 128)  # Resize mel images
CHANNELS = 3           # RGB

# ----------------------------------------------------------------------
# 2️⃣ Load matched MFCC + Mel spectrogram pairs
# ----------------------------------------------------------------------
mfcc_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

mfcc_list, mel_list, labels, speakers = [], [], [], []
missing = 0

for fname in mfcc_files:
    # Example filename:
    # p001_emo_adoration_freeform_aug_noise_seg0.npy
    parts = fname.replace(".npy", "").split("_")
    speaker = parts[0]

    # Extract core key: emo_adoration_freeform_aug_noise
    core_key = "_".join(parts[1:-1])

    if "emo_" not in core_key:
        continue

    # Extract emotion
    emotion = core_key.split("emo_")[1].split("_")[0]
    if emotion not in selected_emotions:
        continue

    # Matching MEL image path
    mel_name = core_key + "_mel.png"
    mel_path = os.path.join(DATA_ROOT, speaker, mel_name)

    if not os.path.exists(mel_path):
        missing += 1
        continue

    # Load MFCC
    mfcc = np.load(os.path.join(npy_dir, fname))

    # Ensure MFCC shape is (time, features)
    if mfcc.ndim == 2 and mfcc.shape[0] < mfcc.shape[1]:
        mfcc = mfcc.T

    # Load Mel image (FIXED)
    mel_img = load_img(mel_path, target_size=IMG_SIZE)
    mel_arr = img_to_array(mel_img) / 255.0

    mfcc_list.append(mfcc)
    mel_list.append(mel_arr)
    labels.append(emotion)
    speakers.append(speaker)

print(f"✅ Loaded {len(mfcc_list)} matched samples (missing mel: {missing})")

# Convert to arrays
X_mfcc = np.array(mfcc_list, dtype=np.float32)
X_mel = np.array(mel_list, dtype=np.float32)
y = np.array(labels)
speakers = np.array(speakers)

print("MFCC shape:", X_mfcc.shape)
print("Mel shape:", X_mel.shape)

# ----------------------------------------------------------------------
# 3️⃣ Speaker-based Split (Same as before)
# ----------------------------------------------------------------------
train_speakers_male = ['p007','p008','p010','p017','p023','p040','p046','p065','p071','p076','p086','p090','p094','p095','p101']
train_speakers_female = ['p002','p006','p024','p026','p041','p043','p047','p055','p063','p067','p080','p083','p084','p104','p098']
val_speakers_male = ['p019','p025','p038','p039','p102']
val_speakers_female = ['p027','p034','p037','p052','p079']
test_speakers_male = ['p001','p074','p081','p085','p105']
test_speakers_female = ['p012','p036','p061','p068','p073']

train_speakers = train_speakers_male + train_speakers_female
val_speakers = val_speakers_male + val_speakers_female
test_speakers = test_speakers_male + test_speakers_female

train_idx = np.isin(speakers, train_speakers)
val_idx = np.isin(speakers, val_speakers)
test_idx = np.isin(speakers, test_speakers)

X_mfcc_train, X_mfcc_val, X_mfcc_test = X_mfcc[train_idx], X_mfcc[val_idx], X_mfcc[test_idx]
X_mel_train, X_mel_val, X_mel_test = X_mel[train_idx], X_mel[val_idx], X_mel[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

print(f"Train / Val / Test samples: {X_mfcc_train.shape[0]}, {X_mfcc_val.shape[0]}, {X_mfcc_test.shape[0]}")
print(f"Train / Val / Test samples: {X_mel_train.shape[0]}, {X_mel_val.shape[0]}, {X_mel_test.shape[0]}")


# ----------------------------------------------------------------------
# 5️⃣ Standardization (Sir’s Style)
# ----------------------------------------------------------------------
scaler = StandardScaler()

X_mfcc_train_sq = np.squeeze(X_mfcc_train)
X_mfcc_val_sq = np.squeeze(X_mfcc_val)
X_mfcc_test_sq = np.squeeze(X_mfcc_test)

scaler.fit(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1]))

X_mfcc_train_scaled = scaler.transform(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1])).reshape(X_mfcc_train_sq.shape)
X_mfcc_val_scaled = scaler.transform(X_mfcc_val_sq.reshape(-1, X_mfcc_val_sq.shape[-1])).reshape(X_mfcc_val_sq.shape)
X_mfcc_test_scaled = scaler.transform(X_mfcc_test_sq.reshape(-1, X_mfcc_test_sq.shape[-1])).reshape(X_mfcc_test_sq.shape)

X_train_st = X_mfcc_train_scaled[..., np.newaxis]
X_val_st = X_mfcc_val_scaled[..., np.newaxis]
X_test_st = X_mfcc_test_scaled[..., np.newaxis]

# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (MFCC)
# ----------------------------------------------------------------------
augmented_segments_mfcc = []
augmented_labels_mfcc = []
aug_factor1 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_train_st.shape[0]):
    original = X_train_st[i]

    # keep original
    augmented_segments_mfcc.append(original)
    augmented_labels_mfcc.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor1):
        aug = random_augment(original)
        augmented_segments_mfcc.append(aug)
        augmented_labels_mfcc.append(y_train[i])

# convert to numpy array
X_train_st_aug = np.array(augmented_segments_mfcc)
y_train_aug = np.array(augmented_labels_mfcc)

print(f"Original train samples: {X_train_st.shape[0]}")
print(f"Augmented train samples: {X_train_st_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_train_st_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (Mel)
# ----------------------------------------------------------------------
augmented_segments_mel = []
augmented_labels_mel = []
aug_factor2 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_mel_train.shape[0]):
    original = X_mel_train[i]

    # keep original
    augmented_segments_mel.append(original)
    augmented_labels_mel.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor2):
        aug = random_augment(original)
        augmented_segments_mel.append(aug)
        augmented_labels_mel.append(y_train[i])

# convert to numpy array
X_mel_train_aug = np.array(augmented_segments_mel)

print(f"Original train samples: {X_mel_train.shape[0]}")
print(f"Augmented train samples: {X_mel_train_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_mel_train_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 4️⃣ Label Encoding
# ----------------------------------------------------------------------
emotion_encoder = LabelEncoder()
emotion_encoder.fit(np.concatenate([y_train_aug, y_val, y_test]))
num_emotions = len(emotion_encoder.classes_)

y_train_one_hot = to_categorical(emotion_encoder.transform(y_train_aug), num_classes=num_emotions)
y_val_one_hot = to_categorical(emotion_encoder.transform(y_val), num_classes=num_emotions)
y_test_one_hot = to_categorical(emotion_encoder.transform(y_test), num_classes=num_emotions)

# ----------------------------------------------------------------------
# 5️⃣ 2D CNN Model
# ----------------------------------------------------------------------
def mfcc_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(0.5)(x)
    return inputs, x


def mel_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    y = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(inputs)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.2)(y)
    y = Dense(64, activation='relu', kernel_regularizer=l2(l2_lambda))(y)
    y = Dropout(0.5)(y)
    return inputs, y

mfcc_in, mfcc_feat = mfcc_branch(X_train_st_aug.shape[1:])
# Fix the shape for the Mel spectrogram input
mel_in, mel_feat = mel_branch(X_mel_train_aug.shape[1:])  # Remove the last dimension in shape

# Now you can safely concatenate the two branches
merged = Concatenate()([mfcc_feat, mel_feat])
x = Dense(128, activation='relu')(merged)
x = Dropout(0.5)(x)
out = Dense(num_emotions, activation='softmax')(x)
model = Model([mfcc_in, mel_in], out)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------------------------------------------------
# 6️⃣ Train
# ----------------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

history = model.fit(

    [X_train_st_aug, X_mel_train_aug],
    y_train_one_hot,
    validation_data=([X_val_st, X_mel_val], y_val_one_hot),
    batch_size=32,
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ----------------------------------------------------------------------
# 7️⃣ Separate Testing by Gender
# ----------------------------------------------------------------------
gender_map = {spk:0 for spk in (train_speakers_male + val_speakers_male + test_speakers_male)}
gender_map.update({spk:1 for spk in (train_speakers_female + val_speakers_female + test_speakers_female)})

y_test_gender = np.array([gender_map[spk] for spk in speakers[test_idx]])
y_true = np.argmax(y_test_one_hot, axis=1)
y_pred_probs = model.predict([X_test_st, X_mel_test])
y_pred = np.argmax(y_pred_probs, axis=1)

male_mask = (y_test_gender == 0)
female_mask = (y_test_gender == 1)

# ---- Male
y_true_male = y_true[male_mask]
y_pred_male = y_pred[male_mask]
male_acc = np.mean(y_pred_male == y_true_male)
male_precision = precision_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_recall = recall_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_f1 = f1_score(y_true_male, y_pred_male, average='macro', zero_division=0)

print("\n--- 👨 Male Test Results ---")
print(f"Accuracy : {male_acc*100:.2f}%")
print(f"Precision: {male_precision*100:.2f}%")
print(f"Recall   : {male_recall*100:.2f}%")
print(f"F1-score : {male_f1*100:.2f}%")
print(classification_report(y_true_male, y_pred_male, target_names=emotion_encoder.classes_))
cm_male = confusion_matrix(y_true_male, y_pred_male)
plt.figure(figsize=(7,6))
sns.heatmap(cm_male, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👨 Confusion Matrix – Male Test Set")
plt.show()

# ---- Female
y_true_female = y_true[female_mask]
y_pred_female = y_pred[female_mask]
female_acc = np.mean(y_pred_female == y_true_female)
female_precision = precision_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_recall = recall_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_f1 = f1_score(y_true_female, y_pred_female, average='macro', zero_division=0)

print("\n--- 👩 Female Test Results ---")
print(f"Accuracy : {female_acc*100:.2f}%")
print(f"Precision: {female_precision*100:.2f}%")
print(f"Recall   : {female_recall*100:.2f}%")
print(f"F1-score : {female_f1*100:.2f}%")
print(classification_report(y_true_female, y_pred_female, target_names=emotion_encoder.classes_))
cm_female = confusion_matrix(y_true_female, y_pred_female)
plt.figure(figsize=(7,6))
sns.heatmap(cm_female, annot=True, fmt='d', cmap='Reds',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👩 Confusion Matrix – Female Test Set")
plt.show()

# ---- Difference
diff_acc = male_acc - female_acc
print("\n--- 🔍 Performance Difference (Male - Female) ---")
print(f"Accuracy Difference : {diff_acc*100:.2f}%")
print(f"Absolute Accuracy Gap : {abs(diff_acc)*100:.2f}%")

# ---- Overall
overall_acc = np.mean(y_pred == y_true)
overall_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
overall_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
overall_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("\n--- ⚖️ Overall Combined Results ---")
print(f"Overall Accuracy : {overall_acc*100:.2f}%")
print(f"Overall Precision: {overall_precision*100:.2f}%")
print(f"Overall Recall   : {overall_recall*100:.2f}%")
print(f"Overall F1-score : {overall_f1*100:.2f}%")
cm_overall = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm_overall, annot=True, fmt='d', cmap='Purples',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("⚖️ Confusion Matrix – Combined Male + Female Test Set")
plt.show()
# -------------------------------
# 4️⃣ Plot Accuracy and Loss Curves
# -------------------------------
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# **Proposed Model(7 emotions)**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, MaxPooling2D,
                                     GlobalAveragePooling2D, Dense, Dropout, Flatten,Concatenate)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder, StandardScaler
import random

def time_mask(spec, max_mask_size=20):
    spec = spec.copy()
    t = spec.shape[0]
    mask = random.randint(0, max_mask_size)
    t0 = random.randint(0, max(1, t - mask))
    spec[t0:t0 + mask, :] = 0
    return spec

def freq_mask(spec, max_mask_size=10):
    spec = spec.copy()
    f = spec.shape[1]
    mask = random.randint(0, max_mask_size)
    f0 = random.randint(0, max(1, f - mask))
    spec[:, f0:f0 + mask] = 0
    return spec

def gaussian_noise(spec, noise_std=0.01):
    return spec + np.random.normal(0, noise_std, spec.shape)

def random_augment(spec):
    """Apply 1–3 random augmentations."""
    aug = spec.copy()
    ops = []

    # randomly choose augmentation operations
    if random.random() < 0.7:
        ops.append(time_mask)
    if random.random() < 0.7:
        ops.append(freq_mask)
    if random.random() < 0.5:
        ops.append(gaussian_noise)

    # apply operations randomly
    random.shuffle(ops)
    for op in ops:
        aug = op(aug)
    return aug

# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
DATA_ROOT = '../input/spect-image/features_mel_image23'
npy_dir = '../input/ears-ser/mfcc_segments_23emotions'
selected_emotions = ["adoration", "anger", "fear", "neutral", "sadness", "disappointment","pain"]

IMG_SIZE = (128, 128)  # Resize mel images
CHANNELS = 3           # RGB

# ----------------------------------------------------------------------
# 2️⃣ Load matched MFCC + Mel spectrogram pairs
# ----------------------------------------------------------------------
mfcc_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

mfcc_list, mel_list, labels, speakers = [], [], [], []
missing = 0

for fname in mfcc_files:
    # Example filename:
    # p001_emo_adoration_freeform_aug_noise_seg0.npy
    parts = fname.replace(".npy", "").split("_")
    speaker = parts[0]

    # Extract core key: emo_adoration_freeform_aug_noise
    core_key = "_".join(parts[1:-1])

    if "emo_" not in core_key:
        continue

    # Extract emotion
    emotion = core_key.split("emo_")[1].split("_")[0]
    if emotion not in selected_emotions:
        continue

    # Matching MEL image path
    mel_name = core_key + "_mel.png"
    mel_path = os.path.join(DATA_ROOT, speaker, mel_name)

    if not os.path.exists(mel_path):
        missing += 1
        continue

    # Load MFCC
    mfcc = np.load(os.path.join(npy_dir, fname))

    # Ensure MFCC shape is (time, features)
    if mfcc.ndim == 2 and mfcc.shape[0] < mfcc.shape[1]:
        mfcc = mfcc.T

    # Load Mel image (FIXED)
    mel_img = load_img(mel_path, target_size=IMG_SIZE)
    mel_arr = img_to_array(mel_img) / 255.0

    mfcc_list.append(mfcc)
    mel_list.append(mel_arr)
    labels.append(emotion)
    speakers.append(speaker)

print(f"✅ Loaded {len(mfcc_list)} matched samples (missing mel: {missing})")

# Convert to arrays
X_mfcc = np.array(mfcc_list, dtype=np.float32)
X_mel = np.array(mel_list, dtype=np.float32)
y = np.array(labels)
speakers = np.array(speakers)

print("MFCC shape:", X_mfcc.shape)
print("Mel shape:", X_mel.shape)

# ----------------------------------------------------------------------
# 3️⃣ Speaker-based Split (Same as before)
# ----------------------------------------------------------------------
train_speakers_male = ['p007','p008','p010','p017','p023','p040','p046','p065','p071','p076','p086','p090','p094','p095','p101']
train_speakers_female = ['p002','p006','p024','p026','p041','p043','p047','p055','p063','p067','p080','p083','p084','p104','p098']
val_speakers_male = ['p019','p025','p038','p039','p102']
val_speakers_female = ['p027','p034','p037','p052','p079']
test_speakers_male = ['p001','p074','p081','p085','p105']
test_speakers_female = ['p012','p036','p061','p068','p073']

train_speakers = train_speakers_male + train_speakers_female
val_speakers = val_speakers_male + val_speakers_female
test_speakers = test_speakers_male + test_speakers_female

train_idx = np.isin(speakers, train_speakers)
val_idx = np.isin(speakers, val_speakers)
test_idx = np.isin(speakers, test_speakers)

X_mfcc_train, X_mfcc_val, X_mfcc_test = X_mfcc[train_idx], X_mfcc[val_idx], X_mfcc[test_idx]
X_mel_train, X_mel_val, X_mel_test = X_mel[train_idx], X_mel[val_idx], X_mel[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

print(f"Train / Val / Test samples: {X_mfcc_train.shape[0]}, {X_mfcc_val.shape[0]}, {X_mfcc_test.shape[0]}")
print(f"Train / Val / Test samples: {X_mel_train.shape[0]}, {X_mel_val.shape[0]}, {X_mel_test.shape[0]}")


# ----------------------------------------------------------------------
# 5️⃣ Standardization (Sir’s Style)
# ----------------------------------------------------------------------
scaler = StandardScaler()

X_mfcc_train_sq = np.squeeze(X_mfcc_train)
X_mfcc_val_sq = np.squeeze(X_mfcc_val)
X_mfcc_test_sq = np.squeeze(X_mfcc_test)

scaler.fit(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1]))

X_mfcc_train_scaled = scaler.transform(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1])).reshape(X_mfcc_train_sq.shape)
X_mfcc_val_scaled = scaler.transform(X_mfcc_val_sq.reshape(-1, X_mfcc_val_sq.shape[-1])).reshape(X_mfcc_val_sq.shape)
X_mfcc_test_scaled = scaler.transform(X_mfcc_test_sq.reshape(-1, X_mfcc_test_sq.shape[-1])).reshape(X_mfcc_test_sq.shape)

X_train_st = X_mfcc_train_scaled[..., np.newaxis]
X_val_st = X_mfcc_val_scaled[..., np.newaxis]
X_test_st = X_mfcc_test_scaled[..., np.newaxis]


# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (MFCC)
# ----------------------------------------------------------------------
augmented_segments_mfcc = []
augmented_labels_mfcc = []
aug_factor1 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_train_st.shape[0]):
    original = X_train_st[i]

    # keep original
    augmented_segments_mfcc.append(original)
    augmented_labels_mfcc.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor1):
        aug = random_augment(original)
        augmented_segments_mfcc.append(aug)
        augmented_labels_mfcc.append(y_train[i])

# convert to numpy array
X_train_st_aug = np.array(augmented_segments_mfcc)
y_train_aug = np.array(augmented_labels_mfcc)

print(f"Original train samples: {X_train_st.shape[0]}")
print(f"Augmented train samples: {X_train_st_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_train_st_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (Mel)
# ----------------------------------------------------------------------
augmented_segments_mel = []
augmented_labels_mel = []
aug_factor2 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_mel_train.shape[0]):
    original = X_mel_train[i]

    # keep original
    augmented_segments_mel.append(original)
    augmented_labels_mel.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor2):
        aug = random_augment(original)
        augmented_segments_mel.append(aug)
        augmented_labels_mel.append(y_train[i])

# convert to numpy array
X_mel_train_aug = np.array(augmented_segments_mel)

print(f"Original train samples: {X_mel_train.shape[0]}")
print(f"Augmented train samples: {X_mel_train_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_mel_train_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 4️⃣ Label Encoding
# ----------------------------------------------------------------------
emotion_encoder = LabelEncoder()
emotion_encoder.fit(np.concatenate([y_train_aug, y_val, y_test]))
num_emotions = len(emotion_encoder.classes_)

y_train_one_hot = to_categorical(emotion_encoder.transform(y_train_aug), num_classes=num_emotions)
y_val_one_hot = to_categorical(emotion_encoder.transform(y_val), num_classes=num_emotions)
y_test_one_hot = to_categorical(emotion_encoder.transform(y_test), num_classes=num_emotions)

# ----------------------------------------------------------------------
# 5️⃣ 2D CNN Model
# ----------------------------------------------------------------------
def mfcc_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(0.5)(x)
    return inputs, x


def mel_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    y = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(inputs)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.2)(y)
    y = Dense(64, activation='relu', kernel_regularizer=l2(l2_lambda))(y)
    y = Dropout(0.5)(y)
    return inputs, y

mfcc_in, mfcc_feat = mfcc_branch(X_train_st_aug.shape[1:])
# Fix the shape for the Mel spectrogram input
mel_in, mel_feat = mel_branch(X_mel_train_aug.shape[1:])  # Remove the last dimension in shape

# Now you can safely concatenate the two branches
merged = Concatenate()([mfcc_feat, mel_feat])
x = Dense(128, activation='relu')(merged)
x = Dropout(0.5)(x)
out = Dense(num_emotions, activation='softmax')(x)
model = Model([mfcc_in, mel_in], out)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------------------------------------------------
# 6️⃣ Train
# ----------------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

history = model.fit(

    [X_train_st_aug, X_mel_train_aug],
    y_train_one_hot,
    validation_data=([X_val_st, X_mel_val], y_val_one_hot),
    batch_size=32,
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ----------------------------------------------------------------------
# 7️⃣ Separate Testing by Gender
# ----------------------------------------------------------------------
gender_map = {spk:0 for spk in (train_speakers_male + val_speakers_male + test_speakers_male)}
gender_map.update({spk:1 for spk in (train_speakers_female + val_speakers_female + test_speakers_female)})

y_test_gender = np.array([gender_map[spk] for spk in speakers[test_idx]])
y_true = np.argmax(y_test_one_hot, axis=1)
y_pred_probs = model.predict([X_test_st, X_mel_test])
y_pred = np.argmax(y_pred_probs, axis=1)

male_mask = (y_test_gender == 0)
female_mask = (y_test_gender == 1)

# ---- Male
y_true_male = y_true[male_mask]
y_pred_male = y_pred[male_mask]
male_acc = np.mean(y_pred_male == y_true_male)
male_precision = precision_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_recall = recall_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_f1 = f1_score(y_true_male, y_pred_male, average='macro', zero_division=0)

print("\n--- 👨 Male Test Results ---")
print(f"Accuracy : {male_acc*100:.2f}%")
print(f"Precision: {male_precision*100:.2f}%")
print(f"Recall   : {male_recall*100:.2f}%")
print(f"F1-score : {male_f1*100:.2f}%")
print(classification_report(y_true_male, y_pred_male, target_names=emotion_encoder.classes_))
cm_male = confusion_matrix(y_true_male, y_pred_male)
plt.figure(figsize=(7,6))
sns.heatmap(cm_male, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👨 Confusion Matrix – Male Test Set")
plt.show()

# ---- Female
y_true_female = y_true[female_mask]
y_pred_female = y_pred[female_mask]
female_acc = np.mean(y_pred_female == y_true_female)
female_precision = precision_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_recall = recall_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_f1 = f1_score(y_true_female, y_pred_female, average='macro', zero_division=0)

print("\n--- 👩 Female Test Results ---")
print(f"Accuracy : {female_acc*100:.2f}%")
print(f"Precision: {female_precision*100:.2f}%")
print(f"Recall   : {female_recall*100:.2f}%")
print(f"F1-score : {female_f1*100:.2f}%")
print(classification_report(y_true_female, y_pred_female, target_names=emotion_encoder.classes_))
cm_female = confusion_matrix(y_true_female, y_pred_female)
plt.figure(figsize=(7,6))
sns.heatmap(cm_female, annot=True, fmt='d', cmap='Reds',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👩 Confusion Matrix – Female Test Set")
plt.show()

# ---- Difference
diff_acc = male_acc - female_acc
print("\n--- 🔍 Performance Difference (Male - Female) ---")
print(f"Accuracy Difference : {diff_acc*100:.2f}%")
print(f"Absolute Accuracy Gap : {abs(diff_acc)*100:.2f}%")

# ---- Overall
overall_acc = np.mean(y_pred == y_true)
overall_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
overall_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
overall_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("\n--- ⚖️ Overall Combined Results ---")
print(f"Overall Accuracy : {overall_acc*100:.2f}%")
print(f"Overall Precision: {overall_precision*100:.2f}%")
print(f"Overall Recall   : {overall_recall*100:.2f}%")
print(f"Overall F1-score : {overall_f1*100:.2f}%")
cm_overall = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm_overall, annot=True, fmt='d', cmap='Purples',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("⚖️ Confusion Matrix – Combined Male + Female Test Set")
plt.show()
# -------------------------------
# 4️⃣ Plot Accuracy and Loss Curves
# -------------------------------
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# **Proposed Model(23 emotions)**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, MaxPooling2D,
                                     GlobalAveragePooling2D, Dense, Dropout, Flatten,Concatenate)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder, StandardScaler


import random

def time_mask(spec, max_mask_size=20):
    spec = spec.copy()
    t = spec.shape[0]
    mask = random.randint(0, max_mask_size)
    t0 = random.randint(0, max(1, t - mask))
    spec[t0:t0 + mask, :] = 0
    return spec

def freq_mask(spec, max_mask_size=10):
    spec = spec.copy()
    f = spec.shape[1]
    mask = random.randint(0, max_mask_size)
    f0 = random.randint(0, max(1, f - mask))
    spec[:, f0:f0 + mask] = 0
    return spec

def gaussian_noise(spec, noise_std=0.01):
    return spec + np.random.normal(0, noise_std, spec.shape)

def random_augment(spec):
    """Apply 1–3 random augmentations."""
    aug = spec.copy()
    ops = []

    # randomly choose augmentation operations
    if random.random() < 0.7:
        ops.append(time_mask)
    if random.random() < 0.7:
        ops.append(freq_mask)
    if random.random() < 0.5:
        ops.append(gaussian_noise)

    # apply operations randomly
    random.shuffle(ops)
    for op in ops:
        aug = op(aug)
    return aug

# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# 1️⃣ Paths & Parameters
# ----------------------------------------------------------------------
DATA_ROOT = '../input/spect-image/features_mel_image23'
npy_dir = '../input/ears-ser/mfcc_segments_23emotions'
selected_emotions = ["adoration", "anger", "amazement", "amusement", "confusion","contentment", "cuteness", "desire", "disappointment", "disgust","distress", "embarassment","extasy", "fear", "guilt","interest", "neutral", "sadness","pain","pride","realization","relief", "serenity"]

IMG_SIZE = (128, 128)  # Resize mel images
CHANNELS = 3           # RGB

# ----------------------------------------------------------------------
# 2️⃣ Load matched MFCC + Mel spectrogram pairs
# ----------------------------------------------------------------------
mfcc_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

mfcc_list, mel_list, labels, speakers = [], [], [], []
missing = 0

for fname in mfcc_files:
    # Example filename:
    # p001_emo_adoration_freeform_aug_noise_seg0.npy
    parts = fname.replace(".npy", "").split("_")
    speaker = parts[0]

    # Extract core key: emo_adoration_freeform_aug_noise
    core_key = "_".join(parts[1:-1])

    if "emo_" not in core_key:
        continue

    # Extract emotion
    emotion = core_key.split("emo_")[1].split("_")[0]
    if emotion not in selected_emotions:
        continue

    # Matching MEL image path
    mel_name = core_key + "_mel.png"
    mel_path = os.path.join(DATA_ROOT, speaker, mel_name)

    if not os.path.exists(mel_path):
        missing += 1
        continue

    # Load MFCC
    mfcc = np.load(os.path.join(npy_dir, fname))

    # Ensure MFCC shape is (time, features)
    if mfcc.ndim == 2 and mfcc.shape[0] < mfcc.shape[1]:
        mfcc = mfcc.T

    # Load Mel image (FIXED)
    mel_img = load_img(mel_path, target_size=IMG_SIZE)
    mel_arr = img_to_array(mel_img) / 255.0

    mfcc_list.append(mfcc)
    mel_list.append(mel_arr)
    labels.append(emotion)
    speakers.append(speaker)

print(f"✅ Loaded {len(mfcc_list)} matched samples (missing mel: {missing})")

# Convert to arrays
X_mfcc = np.array(mfcc_list, dtype=np.float32)
X_mel = np.array(mel_list, dtype=np.float32)
y = np.array(labels)
speakers = np.array(speakers)

print("MFCC shape:", X_mfcc.shape)
print("Mel shape:", X_mel.shape)

# ----------------------------------------------------------------------
# 3️⃣ Speaker-based Split (Same as before)
# ----------------------------------------------------------------------
train_speakers_male = ['p007','p008','p010','p017','p023','p040','p046','p065','p071','p076','p086','p090','p094','p095','p101']
train_speakers_female = ['p002','p006','p024','p026','p041','p043','p047','p055','p063','p067','p080','p083','p084','p104','p098']
val_speakers_male = ['p019','p025','p038','p039','p102']
val_speakers_female = ['p027','p034','p037','p052','p079']
test_speakers_male = ['p001','p074','p081','p085','p105']
test_speakers_female = ['p012','p036','p061','p068','p073']

train_speakers = train_speakers_male + train_speakers_female
val_speakers = val_speakers_male + val_speakers_female
test_speakers = test_speakers_male + test_speakers_female

train_idx = np.isin(speakers, train_speakers)
val_idx = np.isin(speakers, val_speakers)
test_idx = np.isin(speakers, test_speakers)

X_mfcc_train, X_mfcc_val, X_mfcc_test = X_mfcc[train_idx], X_mfcc[val_idx], X_mfcc[test_idx]
X_mel_train, X_mel_val, X_mel_test = X_mel[train_idx], X_mel[val_idx], X_mel[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

print(f"Train / Val / Test samples: {X_mfcc_train.shape[0]}, {X_mfcc_val.shape[0]}, {X_mfcc_test.shape[0]}")
print(f"Train / Val / Test samples: {X_mel_train.shape[0]}, {X_mel_val.shape[0]}, {X_mel_test.shape[0]}")


# ----------------------------------------------------------------------
# 5️⃣ Standardization (Sir’s Style)
# ----------------------------------------------------------------------
scaler = StandardScaler()

X_mfcc_train_sq = np.squeeze(X_mfcc_train)
X_mfcc_val_sq = np.squeeze(X_mfcc_val)
X_mfcc_test_sq = np.squeeze(X_mfcc_test)

scaler.fit(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1]))

X_mfcc_train_scaled = scaler.transform(X_mfcc_train_sq.reshape(-1, X_mfcc_train_sq.shape[-1])).reshape(X_mfcc_train_sq.shape)
X_mfcc_val_scaled = scaler.transform(X_mfcc_val_sq.reshape(-1, X_mfcc_val_sq.shape[-1])).reshape(X_mfcc_val_sq.shape)
X_mfcc_test_scaled = scaler.transform(X_mfcc_test_sq.reshape(-1, X_mfcc_test_sq.shape[-1])).reshape(X_mfcc_test_sq.shape)

X_train_st = X_mfcc_train_scaled[..., np.newaxis]
X_val_st = X_mfcc_val_scaled[..., np.newaxis]
X_test_st = X_mfcc_test_scaled[..., np.newaxis]


# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (MFCC)
# ----------------------------------------------------------------------
augmented_segments_mfcc = []
augmented_labels_mfcc = []
aug_factor1 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_train_st.shape[0]):
    original = X_train_st[i]

    # keep original
    augmented_segments_mfcc.append(original)
    augmented_labels_mfcc.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor1):
        aug = random_augment(original)
        augmented_segments_mfcc.append(aug)
        augmented_labels_mfcc.append(y_train[i])

# convert to numpy array
X_train_st_aug = np.array(augmented_segments_mfcc)
y_train_aug = np.array(augmented_labels_mfcc)

print(f"Original train samples: {X_train_st.shape[0]}")
print(f"Augmented train samples: {X_train_st_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_train_st_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 5️⃣ Apply SpecAugment-style augmentation (Mel)
# ----------------------------------------------------------------------
augmented_segments_mel = []
augmented_labels_mel = []
aug_factor2 = 2  # number of augmented copies per sample (start with 1)

print("Applying augmentation...")

for i in range(X_mel_train.shape[0]):
    original = X_mel_train[i]

    # keep original
    augmented_segments_mel.append(original)
    augmented_labels_mel.append(y_train[i])

    # generate augmented versions
    for _ in range(aug_factor2):
        aug = random_augment(original)
        augmented_segments_mel.append(aug)
        augmented_labels_mel.append(y_train[i])

# convert to numpy array
X_mel_train_aug = np.array(augmented_segments_mel)

print(f"Original train samples: {X_mel_train.shape[0]}")
print(f"Augmented train samples: {X_mel_train_aug.shape[0]}")

# Update label one-hot encoding after augmentation

print(f"\nAfter normalization:")
print(f"Train: {X_train_st.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")
print(f"\nAfter augmentation:")
print(f"Train: {X_mel_train_aug.shape}, Val: {X_val_st.shape}, Test: {X_test_st.shape}")

# ----------------------------------------------------------------------
# 4️⃣ Label Encoding
# ----------------------------------------------------------------------
emotion_encoder = LabelEncoder()
emotion_encoder.fit(np.concatenate([y_train_aug, y_val, y_test]))
num_emotions = len(emotion_encoder.classes_)

y_train_one_hot = to_categorical(emotion_encoder.transform(y_train_aug), num_classes=num_emotions)
y_val_one_hot = to_categorical(emotion_encoder.transform(y_val), num_classes=num_emotions)
y_test_one_hot = to_categorical(emotion_encoder.transform(y_test), num_classes=num_emotions)

# ----------------------------------------------------------------------
# 5️⃣ 2D CNN Model
# ----------------------------------------------------------------------
def mfcc_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda))(x)
    x = Dropout(0.5)(x)
    return inputs, x


def mel_branch(input_shape, l2_lambda=0.01):
    inputs = Input(shape=input_shape)

    y = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(inputs)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = MaxPooling2D((2,2))(y)

    y = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(y)
    y = BatchNormalization()(y)
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.2)(y)
    y = Dense(64, activation='relu', kernel_regularizer=l2(l2_lambda))(y)
    y = Dropout(0.5)(y)
    return inputs, y

mfcc_in, mfcc_feat = mfcc_branch(X_train_st_aug.shape[1:])
# Fix the shape for the Mel spectrogram input
mel_in, mel_feat = mel_branch(X_mel_train_aug.shape[1:])  # Remove the last dimension in shape

# Now you can safely concatenate the two branches
merged = Concatenate()([mfcc_feat, mel_feat])
x = Dense(128, activation='relu')(merged)
x = Dropout(0.5)(x)
out = Dense(num_emotions, activation='softmax')(x)
model = Model([mfcc_in, mel_in], out)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------------------------------------------------
# 6️⃣ Train
# ----------------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

history = model.fit(

    [X_train_st_aug, X_mel_train_aug],
    y_train_one_hot,
    validation_data=([X_val_st, X_mel_val], y_val_one_hot),
    batch_size=32,
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ----------------------------------------------------------------------
# 7️⃣ Separate Testing by Gender
# ----------------------------------------------------------------------
gender_map = {spk:0 for spk in (train_speakers_male + val_speakers_male + test_speakers_male)}
gender_map.update({spk:1 for spk in (train_speakers_female + val_speakers_female + test_speakers_female)})

y_test_gender = np.array([gender_map[spk] for spk in speakers[test_idx]])
y_true = np.argmax(y_test_one_hot, axis=1)
y_pred_probs = model.predict([X_test_st, X_mel_test])
y_pred = np.argmax(y_pred_probs, axis=1)

male_mask = (y_test_gender == 0)
female_mask = (y_test_gender == 1)

# ---- Male
y_true_male = y_true[male_mask]
y_pred_male = y_pred[male_mask]
male_acc = np.mean(y_pred_male == y_true_male)
male_precision = precision_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_recall = recall_score(y_true_male, y_pred_male, average='macro', zero_division=0)
male_f1 = f1_score(y_true_male, y_pred_male, average='macro', zero_division=0)

print("\n--- 👨 Male Test Results ---")
print(f"Accuracy : {male_acc*100:.2f}%")
print(f"Precision: {male_precision*100:.2f}%")
print(f"Recall   : {male_recall*100:.2f}%")
print(f"F1-score : {male_f1*100:.2f}%")
print(classification_report(y_true_male, y_pred_male, target_names=emotion_encoder.classes_))
cm_male = confusion_matrix(y_true_male, y_pred_male)
plt.figure(figsize=(7,6))
sns.heatmap(cm_male, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👨 Confusion Matrix – Male Test Set")
plt.show()

# ---- Female
y_true_female = y_true[female_mask]
y_pred_female = y_pred[female_mask]
female_acc = np.mean(y_pred_female == y_true_female)
female_precision = precision_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_recall = recall_score(y_true_female, y_pred_female, average='macro', zero_division=0)
female_f1 = f1_score(y_true_female, y_pred_female, average='macro', zero_division=0)

print("\n--- 👩 Female Test Results ---")
print(f"Accuracy : {female_acc*100:.2f}%")
print(f"Precision: {female_precision*100:.2f}%")
print(f"Recall   : {female_recall*100:.2f}%")
print(f"F1-score : {female_f1*100:.2f}%")
print(classification_report(y_true_female, y_pred_female, target_names=emotion_encoder.classes_))
cm_female = confusion_matrix(y_true_female, y_pred_female)
plt.figure(figsize=(7,6))
sns.heatmap(cm_female, annot=True, fmt='d', cmap='Reds',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("👩 Confusion Matrix – Female Test Set")
plt.show()

# ---- Difference
diff_acc = male_acc - female_acc
print("\n--- 🔍 Performance Difference (Male - Female) ---")
print(f"Accuracy Difference : {diff_acc*100:.2f}%")
print(f"Absolute Accuracy Gap : {abs(diff_acc)*100:.2f}%")

# ---- Overall
overall_acc = np.mean(y_pred == y_true)
overall_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
overall_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
overall_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("\n--- ⚖️ Overall Combined Results ---")
print(f"Overall Accuracy : {overall_acc*100:.2f}%")
print(f"Overall Precision: {overall_precision*100:.2f}%")
print(f"Overall Recall   : {overall_recall*100:.2f}%")
print(f"Overall F1-score : {overall_f1*100:.2f}%")
cm_overall = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm_overall, annot=True, fmt='d', cmap='Purples',
            xticklabels=emotion_encoder.classes_, yticklabels=emotion_encoder.classes_)
plt.title("⚖️ Confusion Matrix – Combined Male + Female Test Set")
plt.show()
# -------------------------------
# 4️⃣ Plot Accuracy and Loss Curves
# -------------------------------
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()